In [1]:
"""
3 ways to do SentenceLDA:
-> Do Vanilla-LDA: Find weighting for documents. Find word->topic, average over sentences
-> Do Vanilla-LDA where document = sentence, average across sentence for the document
-> ??
"""

'\n3 ways to do SentenceLDA:\n-> Do Vanilla-LDA: Find weighting for documents. Find word->topic, average over sentences\n-> Do Vanilla-LDA where document = sentence, average across sentence for the document\n-> ??\n'

In [1]:
import pickle
import pandas as pd
import re
from nltk.tokenize import RegexpTokenizer

import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.ldamodel import LdaModel

from gensim.models import Phrases
from gensim.parsing.preprocessing import preprocess_string, remove_stopwords, strip_numeric, strip_punctuation, strip_short, stem_text
import matplotlib.pyplot as plt

from gensim import corpora
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary

In [2]:
path = "/mnt/nfs/scratch1/hshukla/"
file = "processed_data_withpunc.csv"
data = pd.read_csv(path + file)

In [3]:
data.head()

,Unnamed: 0,cik,ticker_x,filing_date,item1a_risk,item7_mda,year_x,filing_year_x,perm_id,ticker_y,year_y,company_name,is_dividend_payer,dps_change,is_dps_cut,z_environmental,d_environmental,sector,filing_year_y
0,0,1750,AIR,2010-07-16,item risk factor the follow descript princip r...,item management' discuss and analysi financi c...,2010,2010,4332,AIR,2010,AAR CORP,0.0,2.000000,NaN,NaN,NaN,Industrials,2010
1,1,2034,ACET,2010-09-10,>item risk factor you carefulli consid follow ...,>item manag discuss analysi financi condit res...,2010,2010,2883,ACET,2010,ACETO CORP,1.0,0.000000,0.0,NaN,NaN,Industrials,2010
2,2,2491,BYI,2010-08-26,item risk factor our busi invest common stock ...,item management' discuss and analysi financi c...,2010,2010,2791,BYI,2010,BALLY TECHNOLOGIES INC,0.0,NaN,NaN,NaN,NaN,Tech,2010
3,3,2969,APD,2010-11-23,>item risk factor you read follow risk factor ...,>item manag discuss and analysi financi condit...,2010,2010,2799,APD,2010,AIR PRODUCTS & CHEMICALS INC,1.0,0.149398,0.0,0.620576,-1.857054,Commodities,2010
4,4,3545,ALCO,2010-12-14,>item risk factors. alico oper involv vari deg...,>item manag discuss analysi financi condit res...,2010,2010,1824,ALCO,2010,ALICO INC,1.0,0.181818,0.0,NaN,NaN,Consumer,2010


In [4]:
SELECTED_YEARS = set(list(range(2012,2013)))
display(SELECTED_YEARS)
filtered_by_year = data[data.year_x.isin(SELECTED_YEARS)]
items = {
    'item1a': filtered_by_year['item1a_risk'],
    'item7': filtered_by_year['item7_mda']
}

item_sentences = {}

{2012}

In [5]:
# No bigrams, or lemmanzizing
tokenizer = RegexpTokenizer(r'\w+')
for item_key in items.keys():
    sentences = []
    for idx in range(len(items[item_key])):
        full_string = re.sub("[\(;,\)]", "", items[item_key].iloc[idx].lower())
        these_sentences = full_string.split(".")
        these_sentences = [[word for word in tokenizer.tokenize(sent) if (len(word) > 2 and not word.isnumeric())] for sent in these_sentences]
        items[item_key].iloc[idx] = these_sentences
        
        sentences.extend(these_sentences)
    
    item_sentences[item_key] = sentences

/home/hshukla/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/home/hshukla/anaconda3/lib/python3.8/site-packages/pandas/core/internals/blocks.py:849: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr_value = np.array(value)
<ipython-input-5-03804822a0b6>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

In [6]:
display(len(item_sentences["item1a"]))
display(item_sentences["item1a"][0])
display(len(item_sentences["item1a"][1]))

681041

['item',
 'risk',
 'factor',
 'the',
 'follow',
 'descript',
 'princip',
 'risk',
 'inher',
 'business']

7

In [7]:
dictionaries = {}
for item in item_sentences:
    dictionaries[item] = Dictionary(item_sentences[item])

In [8]:
corpus = {}
for item in item_sentences:
    corpus[item] = [dictionaries[item].doc2bow(doc) for doc in item_sentences[item]]

In [9]:
for item in item_sentences:
    print(item + ':')
    print('\tNumber of unique tokens: %d' % len(dictionaries[item]))
    print('\tNumber of documents: %d' % len(corpus[item]))

item1a:
	Number of unique tokens: 41186
	Number of documents: 681041
item7:
	Number of unique tokens: 59444
	Number of documents: 997904


In [10]:
id2word = {}
for item in items:
    temp = dictionaries[item][0] # Initialize id2token mappings
    id2word[item] = dictionaries[item].id2token

In [11]:
import pickle

base_name = "{}_{}".format(min(list(SELECTED_YEARS)), max(list(SELECTED_YEARS))) + "_{}_{}.pkl"
str_mapping = {
    'corpus': corpus,
    'id2word': id2word
}

for item in items:
    for obj in str_mapping:
        display(item + "_" + obj)
        with open(path+base_name.format(item, obj), 'wb') as file:
            pickle.dump(str_mapping[obj][item], file)

'item1a_corpus'

'item1a_id2word'

'item7_corpus'

'item7_id2word'

In [14]:
params = {
    'num_topics': 30,
    'chunksize': 2000,
    'passes': 20,
    'iterations': 400,
    'eval_every': None,
    'alpha': 'asymmetric',
    'eta': 'auto'
}

In [15]:
models = {}
for item in items:
    display(item)
    models[item] = LdaMulticore(
        corpus=corpus[item],
        id2word=id2word[item],
        workers=32,
        **params
    )

'item1a'

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/hshukla/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/hshukla/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/hshukla/anaconda3/lib/python3.8/multiprocessing/pool.py", line 513, in _handle_workers
    cls._maintain_pool(ctx, Process, processes, pool, inqueue,
  File "/home/hshukla/anaconda3/lib/python3.8/multiprocessing/pool.py", line 337, in _maintain_pool
    Pool._repopulate_pool_static(ctx, Process, processes, pool,
  File "/home/hshukla/anaconda3/lib/python3.8/multiprocessing/pool.py", line 326, in _repopulate_pool_static
    w.start()
  File "/home/hshukla/anaconda3/lib/python3.8/multiprocessing/process.py", line 121, in start
    self._popen = self._Popen(self)
  File "/home/hshukla/anaconda3/lib/python3.8/multiprocessing/context.py", line 277, in _Popen
    return Popen(process_obj)

In [13]:
with open(path+"items", 'wb') as file:
    pickle.dump(items, file)

In [ ]:
display("")

# Sen LDA Explor

In [2]:
path = "/mnt/nfs/scratch1/hshukla/"
start_year = 2012
end_year = 2016
item = "item1full"

base_name = "{}_{}_{}".format(start_year, end_year, item) 

In [3]:
itemLDA = LdaModel.load(path + base_name)

In [4]:
itemLDA.show_topics()

[(0,
  '0.020*"servic" + 0.018*"busi" + 0.017*"new" + 0.013*"abil" + 0.013*"manag" + 0.012*"product" + 0.012*"custom" + 0.012*"our" + 0.012*"market" + 0.010*"provid"'),
 (1,
  '0.026*"regul" + 0.024*"law" + 0.022*"state" + 0.019*"subject" + 0.015*"requir" + 0.015*"unit" + 0.015*"govern" + 0.014*"foreign" + 0.014*"regulatori" + 0.013*"chang"'),
 (2,
  '0.027*"invest" + 0.027*"tax" + 0.018*"asset" + 0.018*"valu" + 0.017*"incom" + 0.014*"acquisit" + 0.011*"rate" + 0.010*"futur" + 0.010*"result" + 0.010*"loss"'),
 (3,
  '0.064*"product" + 0.022*"develop" + 0.019*"manufactur" + 0.018*"approv" + 0.018*"clinic" + 0.017*"commerci" + 0.013*"obtain" + 0.013*"requir" + 0.013*"regulatori" + 0.012*"market"'),
 (4,
  '0.021*"properti" + 0.020*"patent" + 0.019*"claim" + 0.014*"intellectu" + 0.014*"protect" + 0.013*"secur" + 0.013*"parti" + 0.013*"liabil" + 0.012*"right" + 0.012*"insur"'),
 (5,
  '0.089*"advers" + 0.077*"result" + 0.064*"financi" + 0.056*"affect" + 0.052*"materi" + 0.046*"oper" + 0.03

In [18]:
import re

In [23]:
topic_nums = []
word_labels = []
word_weights = []

for item in itemLDA.print_topics():
    topic_nums.append(item[0])
    labels = []
    values = []
    for word_value in item[1].split("+"):
        word_value_arr = word_value.strip().split("*")
        values.append(float(word_value_arr[0].strip()))
        labels.append(re.sub("\"", "", word_value_arr[1].strip()))
    word_labels.append(labels)
    word_weights.append(values)
    

In [24]:
topic_nums

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
import matplotlib.plt as plt

In [26]:
topic = topic_nums[0]
labels = word_labels[0]
weights = word_weights[0]

fig = plt.figure()
ax = sns.barplot(x="Words", y="Importance", data=tips)
ax.set_title(topics)
#ax.set_xlabel("Word")
ax.set_xticklabels(labels)
#ax.set_ylabel("Importance")

AttributeError: 'Figure' object has no attribute 'set_title'

<Figure size 432x288 with 0 Axes>

In [ ]:
for topic,labels,weights in zip(topic_nums, word_labels, word_weights):
    fig = plt.figure()
    
    fig.set_title("Topic {}".format(topic))
    fig.set_xlabel("Word")
    fig.set_xticklabels(labels)
    fig.set_ylabel("CIK Index (Sorted)")
    

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(15,20))

In [25]:
word_weights

[[0.02, 0.018, 0.017, 0.013, 0.013, 0.012, 0.012, 0.012, 0.012, 0.01],
 [0.026, 0.024, 0.022, 0.019, 0.015, 0.015, 0.015, 0.014, 0.014, 0.013],
 [0.027, 0.027, 0.018, 0.018, 0.017, 0.014, 0.011, 0.01, 0.01, 0.01],
 [0.064, 0.022, 0.019, 0.018, 0.018, 0.017, 0.013, 0.013, 0.013, 0.012],
 [0.021, 0.02, 0.019, 0.014, 0.014, 0.013, 0.013, 0.013, 0.012, 0.012],
 [0.089, 0.077, 0.064, 0.056, 0.052, 0.046, 0.039, 0.036, 0.036, 0.029],
 [0.031, 0.026, 0.024, 0.018, 0.017, 0.017, 0.015, 0.015, 0.014, 0.014],
 [0.031, 0.023, 0.022, 0.022, 0.019, 0.019, 0.018, 0.015, 0.013, 0.013],
 [0.065, 0.045, 0.033, 0.025, 0.018, 0.016, 0.016, 0.015, 0.011, 0.01],
 [0.026, 0.024, 0.024, 0.023, 0.022, 0.018, 0.017, 0.015, 0.015, 0.014]]

In [21]:
word_labels

[['servic',
  'busi',
  'new',
  'abil',
  'manag',
  'product',
  'custom',
  'our',
  'market',
  'provid'],
 ['regul',
  'law',
  'state',
  'subject',
  'requir',
  'unit',
  'govern',
  'foreign',
  'regulatori',
  'chang'],
 ['invest',
  'tax',
  'asset',
  'valu',
  'incom',
  'acquisit',
  'rate',
  'futur',
  'result',
  'loss'],
 ['product',
  'develop',
  'manufactur',
  'approv',
  'clinic',
  'commerci',
  'obtain',
  'requir',
  'regulatori',
  'market'],
 ['properti',
  'patent',
  'claim',
  'intellectu',
  'protect',
  'secur',
  'parti',
  'liabil',
  'right',
  'insur'],
 ['advers',
  'result',
  'financi',
  'affect',
  'materi',
  'oper',
  'business',
  'operations',
  'effect',
  'condit'],
 ['increas',
  'price',
  'market',
  'cost',
  'revenu',
  'result',
  'econom',
  'reduc',
  'sale',
  'demand'],
 ['report',
  'million',
  'financi',
  'loan',
  'decemb',
  'risk',
  'year',
  'account',
  'real',
  'control'],
 ['stock',
  'common',
  'share',
  'price',

In [7]:
smallLDA = LdaModel.load(path + "2012_2012_item1a")

In [8]:
smallLDA.print_topics()

[(0,
  '0.086*"result" + 0.071*"advers" + 0.046*"financi" + 0.046*"affect" + 0.044*"oper" + 0.038*"materi" + 0.032*"business" + 0.031*"effect" + 0.026*"operations" + 0.022*"impact"'),
 (1,
  '0.035*"market" + 0.029*"risk" + 0.019*"econom" + 0.015*"chang" + 0.014*"the" + 0.013*"price" + 0.013*"factor" + 0.012*"rate" + 0.012*"industri" + 0.010*"oper"'),
 (2,
  '0.050*"product" + 0.021*"develop" + 0.018*"market" + 0.014*"new" + 0.013*"servic" + 0.013*"technolog" + 0.011*"patent" + 0.011*"products" + 0.010*"commerci" + 0.010*"properti"'),
 (3,
  '0.024*"tax" + 0.019*"incom" + 0.017*"asset" + 0.016*"year" + 0.016*"million" + 0.015*"valu" + 0.015*"net" + 0.013*"revenu" + 0.012*"decemb" + 0.010*"period"'),
 (4,
  '0.033*"manag" + 0.031*"busi" + 0.018*"abil" + 0.017*"oper" + 0.015*"our" + 0.014*"acquir" + 0.014*"acquisit" + 0.014*"continu" + 0.013*"personnel" + 0.013*"depend"'),
 (5,
  '0.030*"regul" + 0.027*"law" + 0.025*"state" + 0.020*"subject" + 0.017*"requir" + 0.017*"regulatori" + 0.017*